In [1]:
import boto3
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
#bucket = '20210520-sagemaker-yolo-ddp'
prefix = 'data'

role = sagemaker.get_execution_role()

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

AWS region:us-east-1


In [3]:
image = "yolov5-sagemaker-109-cu100-4"  # Example: mask-rcnn-smdataparallel-sagemaker
tag = "pt1.8"  # Example: pt1.8

In [4]:
!docker system prune -a -f --volumes
!docker volume rm "docker volume ls -q -f dangling=true"

Total reclaimed space: 0B
Error: No such volume: docker volume ls -q -f dangling=true


In [5]:
!pygmentize ./Dockerfile

ARG region

# FROM 763104351884.dkr.ecr.${region}.amazonaws.com/pytorch-training:1.8.1-gpu-py36-cu111-ubuntu18.04
FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04

#RUN cd /root && \
#	git clone --recursive https://github.com/tkazusa/yolov5-sagemaker-distributed-data-parallel.git && \
#    chmod -R 775 yolov5-sagemaker-distributed-data-parallel && \
#	cd yolov5-sagemaker-distributed-data-parallel && \
#	pip install -r requirements.txt &&\
#    pip install -U smdebug
    

# Install python dependenies
COPY requirements.txt .
RUN python -m pip install --upgrade pip
RUN pip install --no-cache -r requirements.txt coremltools onnx gsutil notebook
RUN pip install -U smdebug


In [6]:
!docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [7]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

creating ECR repository : 815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu100-4:pt1.8 
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  11.49MB
Step 1/6 : ARG region
Step 2/6 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
1.7.1-gpu-py36-cu110-ubuntu18.04: Pulling from pytorch-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: Pulling fs layer 
78bd0b24: Pulling fs layer 
6c31766d: Pulling fs layer 
d9c8e84f: Pulling fs layer 
7fdf2ea6: Pulling fs layer 
c41da723: Pulling fs lay

In [8]:
import os
from sagemaker.pytorch import PyTorch
from sagemaker.local import LocalSession

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]

instance_type = "ml.p3.8xlarge"  # Other supported instance type: ml.p3.16xlarge, ml.p4d.24xlarge
instance_count = 1  # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"  # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
job_name = "pytorch-sm-yolo32"

# SageMakerの操作で使用するローカル用セッション
local_session = LocalSession()

In [9]:
docker_image

'815969174475.dkr.ecr.us-east-1.amazonaws.com/yolov5-sagemaker-109-cu100-4:pt1.8'

In [10]:
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    volume_size=700,
    source_dir=".",
    image_uri=docker_image,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.8.1",
    sagemaker_session=sagemaker_session
)

In [11]:
data_uri = "s3://sagemaker-us-east-1-815969174475/coco/"

data_channels = {"data": data_uri}

estimator.fit(inputs=data_channels, job_name=job_name, wait=False)